# Join cat & brand Table

In [97]:
import pandas as pd
from script.tool import ROOT_NFS_DATA

In [98]:
pro_join = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_UAT' / 'product_joined.csv')
pro_join.head(1)

,EID,PRODUCT_NAME,BID,BRAND_NAME,SCID,SUBCAT_NAME,CID,CAT_NAME
0,13001,UV Aqua Rich Watery Essence SPF50+ PA++++,1528,Biore,248,Facial Sunscreen,62,Sun Product


In [37]:
df = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_product_20231018' / 'datas_20231018.csv')
df.head(1)

,file_names,labels,images_path
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...


In [4]:
pro_join_id = pro_join[['EID', 'BID', 'SCID', 'CID']]

In [25]:
df_joined = pd.merge(df, pro_join_id, how="left", left_on=["labels"], right_on=['EID'])
df_joined.head(1)

,file_names,labels,images_path,EID,BID,SUBCAT_ID,CAT_ID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,14624,1428,2542,59


In [21]:
for i in df_joined.keys():
    if df_joined[i].isnull().sum() > 0:
        print(i, df_joined[i].isnull().sum())

In [32]:
df_joined.to_csv(ROOT_NFS_DATA / 'Cosmenet_product_20231018' / 'datas_20231018_merge.csv', index=False)

In [99]:
df_joined_load = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_product_20231018' / 'datas_20231018_merge.csv')
df_joined_load.head(1)

,file_names,labels,images_path,BID,SCID,CID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59


In [100]:
new_data = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_search_image_only.csv')
new_data.head(1)

,images_path,file_names,labels
0,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838


In [101]:
new_data_joined = pd.merge(new_data, pro_join_id, how="left", left_on=["labels"], 
                           right_on=['EID'])
new_data_joined.drop(columns=['EID'], inplace=True)
new_data_joined.head(1)

,images_path,file_names,labels,BID,SCID,CID
0,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838,3908.0,222.0,58.0


In [102]:
new_data_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34334 entries, 0 to 34333
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   images_path  34334 non-null  object 
 1   file_names   34334 non-null  object 
 2   labels       34334 non-null  int64  
 3   BID          34254 non-null  float64
 4   SCID         34254 non-null  float64
 5   CID          34254 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.6+ MB


In [103]:
new_data_joined = new_data_joined.fillna(value=-1).astype({
    'BID':'int', 'SCID':'int', 'CID':'int'})
new_data_joined.head(1)

,images_path,file_names,labels,BID,SCID,CID
0,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838,3908,222,58


In [104]:
new_data_joined.to_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_search_merge_product_info.csv', 
                       index=False)

In [105]:
new_data_merge = pd.read_csv(
    ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_search_merge_product_info.csv')
new_data_merge.head(1)

,images_path,file_names,labels,BID,SCID,CID
0,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838,3908,222,58


In [106]:
old_concat_new = pd.concat([df_joined_load, new_data_merge]).reset_index(drop=True)
old_concat_new.head(1)

,file_names,labels,images_path,BID,SCID,CID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59


In [107]:
old_concat_new.to_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_last_join_2023_12_18.csv'
                      , index=False)

In [108]:
test = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_last_join_2023_12_18.csv')
print(len(test))
test.head(1)

94530


,file_names,labels,images_path,BID,SCID,CID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59


# Clean Image

In [67]:
import pandas as pd
import numpy as np
from script.tool import ROOT_NFS_DATA, scan_directory
from PIL import Image
from tqdm.notebook import tqdm
import os

In [44]:
path_root = '/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat'

In [47]:
df = scan_directory(path_root, 0, 1)
df.head(1)

amount of all image : 34836


,images_path,file_names,labels
0,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838


In [48]:
not_img = []
for n, i in enumerate(tqdm(df['images_path'])):
    try:
        Image.open(i)
    except:
        not_img.append(n)
        print(n, i)

  0%|          | 0/34836 [00:00<?, ?it/s]

15 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-10.jpg
17 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-06.jpg
18 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-09.jpg
19 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-04.jpg
21 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-02.jpg
23 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-01.jpg
25 /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/42716/PregSkin-l-Intensive-Hydrating-Stretch-Mark-Butter-03.jpg

In [52]:
len(df) - len(not_img)

34364

In [54]:
df.drop(not_img).reset_index(drop=True).to_csv(
    '/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/data_search_image_only.csv', index=False)

In [95]:
test = pd.read_csv('/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/data_search_image_only.csv')
test.tail(1)

,images_path,file_names,labels
34333,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...,Be-You-Tiful-Eyeshadow-Palette.jpg,37992


In [86]:
ls_duplicate = set()
for c in tqdm(test['labels'].unique()):
    img_path = test[test['labels'] == c]['images_path'].values
    n = len(img_path)
    for i, p in enumerate(img_path):
        img_i = np.array(Image.open(p))
        for j in range(i+1, n):
            img_j = np.array(Image.open(img_path[j]))
            if np.array_equal(img_i, img_j):
                ls_duplicate.add(img_path[j])
                print(p, img_path[j])

  0%|          | 0/21088 [00:00<?, ?it/s]

/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/31666/ab26670e683e20845ec20bceaf684c60 (1).jpg /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/31666/ab26670e683e20845ec20bceaf684c60.jpg
/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/36130/54e90203cde256ce1bf22dea2716aab9_11zon.jpg /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/36130/54e90203cde256ce1bf22dea2716aab9 (1)_11zon.jpg
/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/32467/0aeb7d76-d531-5e2a-afbf-f0fe2c906c02 (1)_11zon.jpg /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/32467/0aeb7d76-d531-5e2a-afbf-f0fe2c906c02_11zon.jpg
/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/46649/169501714_11zon_11zon.jpg /app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/datas_image_from_db_uat/46649/169501714 (1)_11zon_11zon.jpg
/app/nfs_clientshare/D

In [88]:
len(ls_duplicate)

30

In [93]:
test.drop(test[test['images_path'].isin(ls_duplicate)].index, inplace=True)

In [94]:
test.to_csv(
    '/app/nfs_clientshare/Datasets/Cosmenet_uat_20231108/data_search_image_only.csv', index=False)

# Mix dataset

In [32]:
import pandas as pd
import numpy as np
from script.tool import ROOT_NFS_DATA
import requests

In [33]:
old_data = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_product_20231018' / 'datas_20231018_merge.csv')
old_data.head(1)

,file_names,labels,images_path,BID,SCID,CID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59


In [34]:
new_data = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_image_only.csv')
new_data.head(1)

,file_names,labels,images_path
0,Kiko-Milano-Floral-Wonderland-Nail-Lacquer-001...,46838,/app/nfs_clientshare/Datasets/Cosmenet_uat_202...


In [35]:
link_cat = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_UAT' / 'product_joined.csv')
link_cat.tail(1)

,EID,PRODUCT_NAME,BID,BRAND_NAME,SCID,SUBCAT_NAME,CID,CAT_NAME
30030,49169,Bio Booster Skin Brightening Sun Surum SPF 50 ...,3992,Am Herb,248,Facial Sunscreen,62,Sun Product


## Product

In [36]:
old_labels_product = old_data['labels'].unique()
new_labels_product = new_data['labels'].unique()
link_cat_product = link_cat['EID'].values
len(old_labels_product), len(new_labels_product), len(link_cat_product)

(4178, 21088, 30031)

In [37]:
# all "old_labels_product" in link_cat_product
old_labels_product_not_new_product = old_labels_product[~np.isin(old_labels_product, new_labels_product)]

new_labels_product_in_old_product = new_labels_product[np.isin(new_labels_product, old_labels_product)]
new_labels_product_in_link_cat_not_old_product = new_labels_product[
    np.isin(new_labels_product, link_cat_product) & ~np.isin(new_labels_product, old_labels_product)]
new_labels_product_not_link_cat = new_labels_product[~np.isin(new_labels_product, link_cat_product)]

link_cat_product_not_all = link_cat_product[
    ~np.isin(link_cat_product, new_labels_product) & ~np.isin(link_cat_product, old_labels_product)]

## has product on cosmenet web

In [490]:
import requests
from requests.sessions import Session
import time
from concurrent.futures import ThreadPoolExecutor
from threading import Thread,local

def check_not_found(result, not_in):
    if not_in:
        return "https://www.cosmenet.in.th/product/not-found" not in result
    else:
        return "https://www.cosmenet.in.th/product/not-found" in result

thread_local = local()

def get_session() -> Session:
    if not hasattr(thread_local,'session'):
        thread_local.session = requests.Session()
    return thread_local.session

def download_link(id:int):
    session = get_session()
    url = 'https://www.cosmenet.in.th/product/' + str(id)
    with session.get(url) as response:
        result = response.text
        if check_not_found(result, not_in):
            res.append(id)
        pbar.update(1)

def download_all(ids:list) -> None:
    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(download_link,ids)

In [489]:
# not has
res = []
pbar = tqdm(total=len(old_labels_product_not_new_product))
not_in = False
download_all(old_labels_product_not_new_product)
pbar.close()
print(res)

  0%|          | 0/162 [00:00<?, ?it/s]

[]


In [493]:
# not has
res = []
pbar = tqdm(total=len(new_labels_product_in_old_product))
not_in = False
download_all(new_labels_product_in_old_product)
pbar.close()
print(res)
# [47702, 43038]

  0%|          | 0/4016 [00:00<?, ?it/s]

[47702, 43038]


In [491]:
# not has
res = []
pbar = tqdm(total=len(new_labels_product_in_link_cat_not_old_product))
not_in = False
download_all(new_labels_product_in_link_cat_not_old_product)
pbar.close()
print(res)
# [13824, 1976, 26418, 26846, 26881, 28057, 29939, 31227, 32459, 40853, 41594, 41725, 42120, 45989, 4785, 4831]

  0%|          | 0/17043 [00:00<?, ?it/s]

[40853, 28057, 32459, 9225, 26846, 8030, 26881, 1976, 13824, 26418, 41594, 42120, 4831, 29939, 4785, 31227, 41725, 45989]


In [481]:
# has
res = []
pbar = tqdm(total=len(new_labels_product_not_link_cat))
not_in = True
download_all(new_labels_product_not_link_cat)
pbar.close()
print(res)

  0%|          | 0/29 [00:00<?, ?it/s]

[]


In [492]:
# not has
res = []
pbar = tqdm(total=len(link_cat_product_not_all))
not_in = False
download_all(link_cat_product_not_all)
pbar.close()
print(res)
# [50505, 49053, 7367, 35536, 33241]

  0%|          | 0/8810 [00:00<?, ?it/s]

[50505, 49053, 7367, 35536, 33241]


In [497]:
# check product that not have category
res = []
scid_2 = link_cat[link_cat['SCID'] == -2]['EID'].values
pbar = tqdm(total=len(scid_2))
not_in = True
download_all(scid_2)
pbar.close()
print(res)

  0%|          | 0/19 [00:00<?, ?it/s]

[]


## add tag not have product on web

In [39]:
all_product_not_in_web = [47702, 43038, 40853, 28057, 32459, 9225, 26846, 
                          8030, 26881, 1976, 13824, 26418, 41594, 42120, 
                          4831, 29939, 4785, 31227, 41725, 45989, 50505, 
                          49053, 7367, 35536, 33241] + new_labels_product_not_link_cat.tolist()

In [45]:
latest_data = pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_last_join_2023_11_10.csv')
latest_data.head(1)

,file_names,labels,images_path,BID,SCID,CID
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59


In [46]:
latest_data['Action'] = 1
latest_data.head(1)

,file_names,labels,images_path,BID,SCID,CID,Action
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59,1


In [58]:
latest_data.loc[latest_data['labels'].isin(all_product_not_in_web), 
                ['Action']] = 0
latest_data[latest_data['Action'] == 0].head(1)

,file_names,labels,images_path,BID,SCID,CID,Action
18497,ID47702_14.jpg,47702,/app/nfs_clientshare/Datasets/Cosmenet_product...,3161,2680,61,0


In [59]:
latest_data.to_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_last_join_2023_11_10.csv',
                  index=False)

In [60]:
pd.read_csv(ROOT_NFS_DATA / 'Cosmenet_uat_20231108' / 'data_last_join_2023_11_10.csv').head(1)

,file_names,labels,images_path,BID,SCID,CID,Action
0,14624_14.jpg,14624,/app/nfs_clientshare/Datasets/Cosmenet_product...,1428,2542,59,1
